In [1]:
import numpy as np
import cv2

In [3]:
def erod(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross' or type_of_mask == 'vertline' or type_of_mask == 'horizline'
    result = image.copy()
    if type_of_mask == 'square':
        result = erod_square(result, size)
    elif type_of_mask == 'cross':
        result = erod_cross(result, size)
    elif type_of_mask == 'vertline':
        result = erod_vert_line(result, size)
    elif type_of_mask == 'horizline':
        result = erod_horiz_line(result, size)
    return result

In [4]:
def process_operation(image, i, j, mask, operation):
    offset = int(mask.shape[0] / 2)
    mask_i = 0
    mask_j = 0
    right_border = offset
    if mask.shape[0] % 2 != 0:
        right_border += 1
    if operation == 'erod':
        for k in range(i - offset, i + right_border):
            mask_j = 0
            for l in range(j - offset, j + right_border):
                if mask[mask_i, mask_j] == 1 and image[k, l] != 0:
                    return 255
                mask_j += 1
            mask_i += 1
    elif operation == 'dilat':
        for k in range(i - offset, i + right_border):
            mask_j = 0
            for l in range(j - offset, j + right_border):
                if mask[mask_i, mask_j] == 1 and image[k, l] == 0:
                    return 0
                mask_j += 1
            mask_i += 1
        return 255
    return 0
    

In [5]:
def erod_square(image, size):
    result = image.copy()
    status = True
    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, np.ones((size, size)), 'erod')
    return result

In [6]:
def erod_cross(image, size):
    result = image.copy()
    status = True
    mask = create_cross(size)
    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, mask, 'erod')
    return result

In [50]:
def create_cross(size):
    mask = np.zeros((size, size))
    center_point = int(size / 2)
    for i in range(size):
        mask[center_point, i] = 1
        mask[i, center_point] = 1
        
    return mask

In [8]:
def dilat(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    if type_of_mask == 'square':
        result = dilat_square(result, size)
    elif type_of_mask == 'cross':
        result = dilat_cross(result, size)
    return result

In [9]:
def dilat_square(image, size):
    result = image.copy()
    status = False
    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, np.ones((size, size)), 'dilat')
    return result

In [10]:
def dilat_cross(image, size):
    result = image.copy()
    status = False
    mask = create_cross(size)
    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, mask, 'dilat')
    return result

In [11]:
def opening(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    result = erod(result, size, type_of_mask)
    result = dilat(result, size, type_of_mask)
    return result

In [12]:
def closing(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    result = dilat(result, size, type_of_mask)
    result = erod(result, size, type_of_mask)
    return result

In [80]:
def filter(image):
    result = image.copy()
    for i in range(3, image.shape[0] - 3):
        for j in range(3, image.shape[1] - 3):
            if image[i, j] == 0:
                if image[i, j + 1] == 255 and image[i, j - 1] == 255 and image[i + 1, j] == 255 and image[i - 1, j] == 255:
                    result[i, j] = 255
            elif image[i, j] == 255:
                if image[i, j + 1] == 0 and image[i, j - 1] == 0 and image[i + 1, j] == 0 and image[i - 1, j] == 0:
                    result[i, j] = 0
    return result

In [14]:
def make_noise(image, prob):
    result = image.copy()
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            p = np.random.randint(0, 101)
            if p < prob :
                p_color = np.random.randint(0, 101)
                if p_color < 50:
                    result[i, j] = 0
                else:
                    result[i, j] = 255
    return result

In [15]:
def circle(image, centre, radius):
    circle = [centre, radius]
    for i in range(circle[0][0] - circle[1], circle[0][0] + circle[1]):
        for j in range(circle[0][1] - circle[1], circle[0][1] + circle[1]):
            if (i - circle[0][0]) ** 2 + (j - circle[0][1]) ** 2 < circle[1] ** 2:
                image[i, j] = 0

In [16]:
def triangle(image, A, B, C):
    triangle = [A, B, C]
    x_a = triangle[0][1]
    y_a = triangle[0][0]
    x_b = triangle[1][1]
    y_b = triangle[1][0]
    x_c = triangle[2][1]
    y_c = triangle[2][0]

    left = triangle[0][0]
    right = triangle[0][0]
    above = triangle[0][1]
    below = triangle[0][1]
    for i in triangle:
        if left > i[0]:
            left = i[0]
        if right < i[0]:
            right = i[0]
        if above > i[1]:
            above = i[1]
        if below < i[1]:
            below = i[1]
    for i in range(above, below):
        for j in range(left, right):
            coord_1 = (x_a - i) * (y_b - y_a) - (x_b - x_a) * (y_a - j)
            coord_2 = (x_b - i) * (y_c - y_b) - (x_c - x_b) * (y_b - j)
            coord_3 = (x_c - i) * (y_a - y_c) - (x_a - x_c) * (y_c - j)
            if coord_1 >= 0 and coord_2 >= 0 and coord_3 >= 0:
                image[i, j] = 0

In [17]:
def square(image, A, B):
    square = [A, B]
    for i in range(square[0][0], square[1][0]):
        for j in range(square[0][1], square[1][1]):
            image[i, j] = 0

In [18]:
def noise_reduce(image, filter_image, noised_image):
    assert image.shape == filter_image.shape and image.shape == noised_image.shape
    count_noised = 0
    count_filtered = 0
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i, j] - noised_image[i, j] != 0:
                count_noised += 1
            if image[i, j] - filter_image[i, j] != 0:
                count_filtered += 1
    return count_noised / count_filtered

# 1. Фильтрация бинарных изображений

## 1.1. Создать или выбрать бинарное изображение с любым простым рисунком, состоящим из примитивных фигур (круги, квадраты, полигоны).

In [21]:
image = np.zeros((500, 500))
image.fill(255)

In [22]:
square(image, (10, 10), (50, 50))
triangle(image, (100, 100), (100, 200), (200, 100))
circle(image, (400, 400), 40)
circle(image, (200, 250), 1)

In [23]:
path = '/home/sergei/laba2/'
cv2.imwrite(path + 'image.png', image)

True

## 1.2. На полученное изображение наложить шум (1%, 2%, 5%, 10%, 20%, 50%).

In [24]:
noised = []
for i in [1, 2, 5, 10, 20, 50]:
    noise = make_noise(image, i)
    noised.append(noise)
    cv2.imwrite(path + 'noise/image_noise_' + str(i) + '%' + '.png', noise)

## 1.3. Выполнить операции эрозии и дилатации для различных структурных элементов (крест, квадрат). Визуально оценить результаты эрозии и дилатации.

In [25]:
for i in range(3, 6):
    erod_image = erod(image, i, type_of_mask = 'square')
    cv2.imwrite(path + 'erod/erod_image_square_size' + str(i) + '.png', erod_image)

In [26]:
for i in range(3, 6):
    erod_image = erod(image, i, type_of_mask = 'cross')
    cv2.imwrite(path + 'erod/erod_image_cross_size'+ str(i) + '.png', erod_image)

In [27]:
for i in range(3, 6):
    dilat_image = dilat(image, i, type_of_mask = 'square')
    cv2.imwrite(path + 'dilat/dilat_image_square_size' + str(i) + '.png', dilat_image)

In [28]:
for i in range(3, 6):
    dilat_image= dilat(image, i, type_of_mask = 'cross')
    cv2.imwrite(path + 'dilat/dilat_image_cross_size' + str(i) + '.png', dilat_image)

## 1.4. Провести фильтрацию шумов на изображении при помощи морфологических операций вскрытия и закрытия с различными размерами (от 3 до 5) и формами структурного элемента (крест, квадрат), а также алгоритма фильтрации шумов, которому соответствует логическая таблица истинности, представленная в табл. 1 со крестообразным структурным элементом размером 3.

In [29]:
noise_index = [1, 2, 5, 10, 20, 50]

In [30]:
opened_image_square = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        open_image = opening(noised[j], i, type_of_mask = 'square')
        opened_image_square[i - 3, j] = open_image
        cv2.imwrite(path + 'open/open_image_square_noise' + str(noise_index[j]) + '%_size' + str(i) + '.png', open_image)

In [31]:
closed_image_square = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        close_image = closing(noised[j], i, type_of_mask = 'square')
        closed_image_square[i - 3, j] = close_image
        cv2.imwrite(path + 'close/close_image_square' + str(noise_index[j]) + '%_size' + str(i) + '.png', close_image)

In [32]:
opened_image_cross = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        open_image = opening(noised[j], i, type_of_mask = 'cross')
        opened_image_cross[i - 3, j] = open_image
        cv2.imwrite(path + 'open/open_image_cross' + str(noise_index[j]) + '%_size' + str(i) + '.png', open_image)

In [33]:
closed_image_cross = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        close_image = closing(noised[j], i, type_of_mask = 'cross')
        closed_image_cross[i - 3, j] = close_image
        cv2.imwrite(path + 'close/close_image_cross' + str(noise_index[j]) + '%_size' + str(i) + '.png', close_image)

In [81]:
filtered_image = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(len(noised)):
    filter_image = filter(noised[i])
    filtered_image[i - 3, j] = filter_image
    cv2.imwrite(path + 'filter_image_' + str(i) + '.png', filter_image)

## 1.5. Рассчитать коэффициент снижения шума для различных методов фильтрации, для различной интенсивности шума и различных размеров и форм структурного элемента (окна).

In [35]:
for i in range(opened_image_square.shape[0]):
    for j in range(opened_image_square.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, opened_image_square[i, j], noised[j])))

Size 3 Noised 0 reduce 0.432
Size 3 Noised 1 reduce 0.625
Size 3 Noised 2 reduce 1.0227272727272727
Size 3 Noised 3 reduce 1.599869451697128
Size 3 Noised 4 reduce 2.561256846353416
Size 3 Noised 5 reduce 4.04217791411043
Size 4 Noised 0 reduce 0.05750798722044728
Size 4 Noised 1 reduce 0.1486013986013986
Size 4 Noised 2 reduce 0.4032683183974697
Size 4 Noised 3 reduce 0.7602357320099256
Size 4 Noised 4 reduce 1.2951895043731778
Size 4 Noised 5 reduce 2.906133700895934
Size 5 Noised 0 reduce 0.19424460431654678
Size 5 Noised 1 reduce 0.21992238033635186
Size 5 Noised 2 reduce 0.3541666666666667
Size 5 Noised 3 reduce 0.5428571428571428
Size 5 Noised 4 reduce 0.8464323139944746
Size 5 Noised 5 reduce 2.490873648768386


In [36]:
for i in range(closed_image_square.shape[0]):
    for j in range(closed_image_square.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, closed_image_square[i, j], noised[j])))

Size 3 Noised 0 reduce 0.043866774979691305
Size 3 Noised 1 reduce 0.0660964230171073
Size 3 Noised 2 reduce 0.11018291804695377
Size 3 Noised 3 reduce 0.14506392045454544
Size 3 Noised 4 reduce 0.1956445149072973
Size 3 Noised 5 reduce 0.27234504272345045
Size 4 Noised 0 reduce 0.026277372262773723
Size 4 Noised 1 reduce 0.04788732394366197
Size 4 Noised 2 reduce 0.08099523557437797
Size 4 Noised 3 reduce 0.09152010753892685
Size 4 Noised 4 reduce 0.11147775463601917
Size 4 Noised 5 reduce 0.20332020231729486
Size 5 Noised 0 reduce 0.03907380607814761
Size 5 Noised 1 reduce 0.0545046489259378
Size 5 Noised 2 reduce 0.0619383045907214
Size 5 Noised 3 reduce 0.0556792367105861
Size 5 Noised 4 reduce 0.06979959620717557
Size 5 Noised 5 reduce 0.18781985898366688


In [37]:
for i in range(opened_image_cross.shape[0]):
    for j in range(opened_image_cross.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, opened_image_cross[i, j], noised[j])))

Size 3 Noised 0 reduce 0.5142857142857142
Size 3 Noised 1 reduce 0.8173076923076923
Size 3 Noised 2 reduce 1.383363471971067
Size 3 Noised 3 reduce 2.2221214868540344
Size 3 Noised 4 reduce 3.922737306843267
Size 3 Noised 5 reduce 5.59628400796284
Size 4 Noised 0 reduce 0.0743801652892562
Size 4 Noised 1 reduce 0.21656050955414013
Size 4 Noised 2 reduce 0.7642357642357642
Size 4 Noised 3 reduce 1.7671232876712328
Size 4 Noised 4 reduce 3.3566301473366074
Size 4 Noised 5 reduce 5.022989874925551
Size 5 Noised 0 reduce 0.34177215189873417
Size 5 Noised 1 reduce 0.4899135446685879
Size 5 Noised 2 reduce 0.8306188925081434
Size 5 Noised 3 reduce 1.5083076923076923
Size 5 Noised 4 reduce 2.4653163152053272
Size 5 Noised 5 reduce 3.97886393659181


In [38]:
for i in range(closed_image_cross.shape[0]):
    for j in range(closed_image_cross.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, closed_image_cross[i, j], noised[j])))

Size 3 Noised 0 reduce 0.04488778054862843
Size 3 Noised 1 reduce 0.07203389830508475
Size 3 Noised 2 reduce 0.1302127659574468
Size 3 Noised 3 reduce 0.1986062717770035
Size 3 Noised 4 reduce 0.3217221276749828
Size 3 Noised 5 reduce 0.4454868154158215
Size 4 Noised 0 reduce 0.07659574468085106
Size 4 Noised 1 reduce 0.21464646464646464
Size 4 Noised 2 reduce 0.6057007125890737
Size 4 Noised 3 reduce 0.6780082987551868
Size 4 Noised 4 reduce 0.5485245091986665
Size 4 Noised 5 reduce 0.3934462939464059
Size 5 Noised 0 reduce 0.045416316232127836
Size 5 Noised 1 reduce 0.07154882154882154
Size 5 Noised 2 reduce 0.1291139240506329
Size 5 Noised 3 reduce 0.19559492458702418
Size 5 Noised 4 reduce 0.28728942348110065
Size 5 Noised 5 reduce 0.3142269516229992


In [39]:
for i in range(filtered_image.shape[0]):
    for j in range(filtered_image.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, filtered_image[i, j], noised[j])))

Size 3 Noised 0 reduce 0.0002266650436330209
Size 3 Noised 1 reduce 0.0007135751373632139
Size 3 Noised 2 reduce 0.0032110881181344627
Size 3 Noised 3 reduce 0.010288074480454337
Size 3 Noised 4 reduce 0.037294794679247976
Size 3 Noised 5 reduce 3.4199513381995135
Size 4 Noised 0 reduce 0.0002266650436330209
Size 4 Noised 1 reduce 0.0007135751373632139
Size 4 Noised 2 reduce 0.0032110881181344627
Size 4 Noised 3 reduce 0.010288074480454337
Size 4 Noised 4 reduce 0.037294794679247976
Size 4 Noised 5 reduce 1.7124060913705583
Size 5 Noised 0 reduce 0.0002266650436330209
Size 5 Noised 1 reduce 0.0007135751373632139
Size 5 Noised 2 reduce 0.0032110881181344627
Size 5 Noised 3 reduce 0.010288074480454337
Size 5 Noised 4 reduce 0.037294794679247976
Size 5 Noised 5 reduce 0.6901360043207149


# 2. Выделение контуров на бинарных изображениях

## 2.1. Создать или выбрать бинарное изображение, содержащее простой рисунок, состоящий из примитивных фигур (полосы, круги, квадраты).

In [40]:
image = np.zeros((500, 500))
image.fill(255)

In [41]:
square(image, (10, 10), (50, 50))
triangle(image, (100, 100), (100, 200), (200, 100))
circle(image, (400, 400), 40)
circle(image, (200, 250), 1)

In [42]:
cv2.imwrite(path + 'image.png', image)

True

## 2.2. С помощью морфологических операций выделить контур объекта. Выяснить, когда контур получается внешним, внутренним, четырёхсвязным, восьмисвязным.

In [75]:
def border(image, mask_type):
    assert mask_type == 'square' or mask_type == 'cross' or mask_type == 'vertline' or mask_type == 'horizline'
    
    result = image.copy()
    erod_image = erod(image, 3, mask_type)
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            if image[i, j] - erod_image[i, j] != 0:
                result[i, j] = 0
            else:
                result[i, j] = 255
    return result

In [76]:
b = border(image, 'horizline')
cv2.imwrite(path + 'border_horiz.png', b)

True

In [77]:
def erod_vert_line(image, size):
    result = image.copy()
    status = True
    
    mask = np.zeros((size, size))
    center_point = int(size / 2)
    for i in range(size):
        mask[i, center_point] = 1

    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, mask, 'erod')
    return result

In [78]:
b = border(image, 'vertline')
cv2.imwrite(path + 'border_vert.png', b)

True

In [79]:
def erod_horiz_line(image, size):
    result = image.copy()
    status = True
    
    mask = np.zeros((size, size))
    center_point = int(size / 2)
    for i in range(size):
        mask[center_point, i] = 1

    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, mask, 'erod')
    return result